In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-it27lpp2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-it27lpp2
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 824.9 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369500 sha256=28cf75d439df8cdd77014cb3b4d1b5875e8983eca08a66172be0cba167c11a6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-wlgq3slr/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from PIL import Image
from glob import glob
from os import path
import torch
import clip
import numpy as np
import pandas as pd
from tqdm import tqdm

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# This is where my dataset lives

In [ ]:
dataset_location = '/content/gdrive/MyDrive/nln-dataset'

# Extract CLIP features

In this step CLIP features are extracted from the images in the dataset. The result is stored in clip-features.csv

In [ ]:
dataset_location_normalized = f"{dataset_location}/normalized"

if not path.exists(dataset_location_normalized):
    print("run NB00_normalize first")

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

to_process = [sorted(glob(f'{label}/*')) for label in sorted(glob(f"{dataset_location_normalized}/*"))]
to_process_flat = [item for lst in to_process for item in lst]

labels = sorted(set([path.dirname(path.relpath(x, dataset_location_normalized)) for x in to_process_flat]))
print(labels)

images = [preprocess(Image.open(i)).unsqueeze(0).to(device) for i in tqdm(to_process_flat)]

print("encoding images")
with torch.no_grad():
    features = [model.encode_image(i).cpu().numpy().reshape(512) for i in images]

fa = np.array(features)

labeling = [labels.index(path.dirname(path.relpath(x, dataset_location_normalized))) for x in to_process_flat]

identifier = [path.relpath(x, dataset_location_normalized) for x in to_process_flat]
identifier_df = pd.DataFrame(identifier)
identifier_df = identifier_df.rename(columns={0: "id"})

label_df = pd.DataFrame(labeling)
label_df = label_df.rename(columns={0: "label"})

df = pd.DataFrame(fa)
df = pd.concat([identifier_df, label_df, df], axis=1)

df.to_csv(f"{dataset_location}/clip-features.csv", index=False)